## Decoradores

In [6]:
def titulo(func): #esta é a chamada do decorador
    def wrapper(): #este é o conteúdo do decorador
        print("# Preço atual do Bitcoin")
        func()
    return wrapper

In [11]:
# Criando uma função
import requests
@titulo
def cotacao_btc():
    link = f'https://economia.awesomeapi.com.br/json/last/BTC-BRL'
    requisicao = requests.get(link)
    requisicao = requisicao.json()
    BTC = requisicao['BTCBRL']['bid']
    print(BTC)

ModuleNotFoundError: No module named 'requests'

In [8]:
cotacao_btc()

NameError: name 'cotacao_btc' is not defined

In [ ]:
from datetime import datetime
from functools import cache
import time
from time import sleep

def medidor_de_tempo(func):
    def aninhada(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fim = time.time()
        tempo = fim - inicio
        print(f'{func.__name__} demorou {round(tempo,3)} segundos.')
        return resultado
    return aninhada

#### Decorator cache

In [ ]:
@cache
@medidor_de_tempo
def delay(secs):
    """bota o tempo pra dormir por 'secs'. """
    sleep(secs)
    return secs

In [ ]:
print(
    delay(1),delay(2),
    delay(1), delay(2)
)

##### Decorators em classes

In [ ]:
class UpperNome(object):
    def __init__(self, f):
        self.f = f
        
    def __call__(self, *args):
        self.f(args[0].upper())
        
    


def GetNome(name):
    print(name)
    
@UpperNome
def GetSobrenome(sobrenome):
    print(sobrenome)
    
    
nome = str(input("Digite seu nome: "))
sobrenome = str(input("Digite seu end:"))
GetNome(nome)
GetSobrenome(sobrenome)


## Programação Reativa

In [ ]:
#!pip install rx

In [ ]:
#programação reativa
import rx
import rx.operators as ops

#source = rx.from_iterable([1,2,3,4,5,10,15])
source = rx.from_iterable([1,2,'abc',4,5]) #observable

disposable=source.pipe(
    ops.map(lambda i:i-1),
    ops.filter(lambda i:i%2==0),
).subscribe( #observer
    
    on_next=lambda i: print("on_next: {} ".format(i)),
    on_completed=lambda: print("on_completed"),
    on_error=lambda e:print("on_error: {}".format(e))
)

disposable.dispose()

print("Fim")

## Concorrencia & Paralelismo

In [ ]:
#Criando threds independentes
import _thread # Método Primitivo
def filho(tid):
    print('Olá pertencente a thread', tid)

def pai():
    i = 0
    while True:
        i += 1
        _thread.start_new_thread(filho, (i,)) #começando uma nova Thread
        if input() == 'q': break

pai()

In [ ]:
# Processso de Paralelismo
import _thread as thread, time
def contador(meuId, cont):
    for i in range(cont):
        time.sleep(1)
        print('[%s] => %s' % (meuId, i))

for i in range(5):
    thread.start_new_thread(contador, (i, 5))

time.sleep(6)
print('Saindo da thread principal.')

In [ ]:
# travando a thread
import _thread as thread, time
def contador(meuId, cont):
    for i in range(cont):
        time.sleep(1)
        mutex.acquire()
        print('[%s] => %s' % (meuId, i))
        mutex.release()

mutex = thread.allocate_lock()
for i in range(5):
    thread.start_new_thread(contador, (i, 5))

time.sleep(6)
print('Saindo da thread principal.')

In [ ]:
# Processo de concorrencia
import _thread as thread

stdoutmutex = thread.allocate_lock()
exitmutexes = [False] * 10

def contador(meuId, cont):
    for i in range(cont):
        stdoutmutex.acquire()
        print('[%s] => %s' % (meuId, i))
        stdoutmutex.release()
    exitmutexes[meuId] = True

for i in range(10):
    thread.start_new_thread(contador, (i, 100))

while False in exitmutexes: pass

print('Saindo da thread principal.')

In [ ]:
import threading

class Minhathread(threading.Thread):
    def __init__(self, meuId, cont, mutex):
        self.meuId = meuId
        self.cont = cont
        self.mutex = mutex
        threading.Thread.__init__(self)
    def run(self):
        for i in range(self.cont):
            with self.mutex: #simiar a usar o acquire e release
                print('[%s] => %s' % (self.meuId, i))

stdoutmutex = threading.Lock()
threads = []

for i in range(10):
    thread = Minhathread(i, 100, stdoutmutex)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join() #Espera até a threading está terminada.

print('Saindo da Thread principal.')


In [ ]:
from random import random

class Produto():
    def __init__(self, nome, espaco, valor):
        self.nome = nome
        self.espaco = espaco
        self.valor = valor
        
class Individuo():
    def __init__(self, espacos, valores, limite_espacos, geracao=0):
        self.espacos = espacos
        self.valores = valores
        self.limite_espacos = limite_espacos
        self.nota_avaliacao = 0
        self.espaco_usado = 0
        self.geracao = geracao
        self.cromossomo = []
        
        for i in range(len(espacos)):
            if random() < 0.5:
                self.cromossomo.append("0")
            else:
                self.cromossomo.append("1")
                
    def avaliacao(self):
        nota = 0
        soma_espacos = 0
        for i in range(len(self.cromossomo)):
           if self.cromossomo[i] == '1':
               nota += self.valores[i]
               soma_espacos += self.espacos[i]
        if soma_espacos > self.limite_espacos:
            nota = 1
        self.nota_avaliacao = nota
        self.espaco_usado = soma_espacos
        
    def crossover(self, outro_individuo):
        corte = round(random()  * len(self.cromossomo))
        
        filho1 = outro_individuo.cromossomo[0:corte] + self.cromossomo[corte::]
        filho2 = self.cromossomo[0:corte] + outro_individuo.cromossomo[corte::]
        
        filhos = [Individuo(self.espacos, self.valores, self.limite_espacos, self.geracao + 1),
                  Individuo(self.espacos, self.valores, self.limite_espacos, self.geracao + 1)]
        filhos[0].cromossomo = filho1
        filhos[1].cromossomo = filho2
        return filhos
    
    def mutacao(self, taxa_mutacao):
        #print("Antes %s " % self.cromossomo)
        for i in range(len(self.cromossomo)):
            if random() < taxa_mutacao:
                if self.cromossomo[i] == '1':
                    self.cromossomo[i] = '0'
                else:
                    self.cromossomo[i] = '1'
        #print("Depois %s " % self.cromossomo)
        return self
        
class AlgoritmoGenetico():
    def __init__(self, tamanho_populacao):
        self.tamanho_populacao = tamanho_populacao
        self.populacao = []
        self.geracao = 0
        self.melhor_solucao = 0
        
    def inicializa_populacao(self, espacos, valores, limite_espacos):
        for i in range(self.tamanho_populacao):
            self.populacao.append(Individuo(espacos, valores, limite_espacos))
        self.melhor_solucao = self.populacao[0]
        
    def ordena_populacao(self):
        self.populacao = sorted(self.populacao,
                                key = lambda populacao: populacao.nota_avaliacao,
                                reverse = True)
        
    def melhor_individuo(self, individuo):
        if individuo.nota_avaliacao > self.melhor_solucao.nota_avaliacao:
            self.melhor_solucao = individuo
            
    def soma_avaliacoes(self):
        soma = 0
        for individuo in self.populacao:
           soma += individuo.nota_avaliacao
        return soma
    
    def seleciona_pai(self, soma_avaliacao):
        pai = -1
        valor_sorteado = random() * soma_avaliacao
        soma = 0
        i = 0
        while i < len(self.populacao) and soma < valor_sorteado:
            soma += self.populacao[i].nota_avaliacao
            pai += 1
            i += 1
        return pai
    
    def visualiza_geracao(self):
        melhor = self.populacao[0]
        print("G:%s -> Valor: %s Espaço: %s Cromossomo: %s" % (self.populacao[0].geracao,
                                                               melhor.nota_avaliacao,
                                                               melhor.espaco_usado,
                                                               melhor.cromossomo))
    
    def resolver(self, taxa_mutacao, numero_geracoes, espacos, valores, limite_espacos):
        self.inicializa_populacao(espacos, valores, limite_espacos)
        
        for individuo in self.populacao:
            individuo.avaliacao()
        
        self.ordena_populacao()
        
        self.visualiza_geracao()
        
        for geracao in range(numero_geracoes):
            soma_avaliacao = self.soma_avaliacoes()
            nova_populacao = []
            
            for individuos_gerados in range(0, self.tamanho_populacao, 2):
                pai1 = self.seleciona_pai(soma_avaliacao)
                pai2 = self.seleciona_pai(soma_avaliacao)
                
                filhos = self.populacao[pai1].crossover(self.populacao[pai2])
                
                nova_populacao.append(filhos[0].mutacao(taxa_mutacao))
                nova_populacao.append(filhos[1].mutacao(taxa_mutacao))
            
            self.populacao = list(nova_populacao)
            
            for individuo in self.populacao:
                individuo.avaliacao()
            
            self.ordena_populacao()
            
            self.visualiza_geracao()
            
            melhor = self.populacao[0]
            self.melhor_individuo(melhor)
        
        print("\nMelhor solução -> G: %s Valor: %s Espaço: %s Cromossomo: %s" %
              (self.melhor_solucao.geracao,
               self.melhor_solucao.nota_avaliacao,
               self.melhor_solucao.espaco_usado,
               self.melhor_solucao.cromossomo))
        
        return self.melhor_solucao.cromossomo
        
        
if __name__ == '__main__':
    #p1 = Produto("Iphone 6", 0.0000899, 2199.12)
    lista_produtos = []
    lista_produtos.append(Produto("Geladeira Dako", 0.751, 999.90))
    lista_produtos.append(Produto("Iphone 6", 0.0000899, 2911.12))
    lista_produtos.append(Produto("TV 55' ", 0.400, 4346.99))
    lista_produtos.append(Produto("TV 50' ", 0.290, 3999.90))
    lista_produtos.append(Produto("TV 42' ", 0.200, 2999.00))
    lista_produtos.append(Produto("Notebook Dell", 0.00350, 2499.90))
    lista_produtos.append(Produto("Ventilador Panasonic", 0.496, 199.90))
    lista_produtos.append(Produto("Microondas Electrolux", 0.0424, 308.66))
    lista_produtos.append(Produto("Microondas LG", 0.0544, 429.90))
    lista_produtos.append(Produto("Microondas Panasonic", 0.0319, 299.29))
    lista_produtos.append(Produto("Geladeira Brastemp", 0.635, 849.00))
    lista_produtos.append(Produto("Geladeira Consul", 0.870, 1199.89))
    lista_produtos.append(Produto("Notebook Lenovo", 0.498, 1999.90))
    lista_produtos.append(Produto("Notebook Asus", 0.527, 3999.00))
    
    espacos = []
    valores = []
    nomes = []
    for produto in lista_produtos:
        espacos.append(produto.espaco)
        valores.append(produto.valor)
        nomes.append(produto.nome)
    limite = 3
    tamanho_populacao = 20
    taxa_mutacao = 0.01
    numero_geracoes = 100
    ag = AlgoritmoGenetico(tamanho_populacao)
    resultado = ag.resolver(taxa_mutacao, numero_geracoes, espacos, valores, limite)
    for i in range(len(lista_produtos)):
        if resultado[i] == '1':
            print("Nome: %s R$ %s " % (lista_produtos[i].nome,
                                       lista_produtos[i].valor))
    
    
    

<b>Links úteis

https://docs.awesomeapi.com.br/api-de-moedas <br>
https://docs.python.org/3/library/functools.html <br>
https://docs.python.org/3/library/threading.html <br>
https://beta.openai.com/playground
